In [ ]:
import tifffile
from pathlib import Path

In [ ]:
DATASET_FOLDER = "/localscratch/calcium_imaging_dataset/calcium_imaging/Test_Data"
dataset_path = Path(DATASET_FOLDER)

In [ ]:
from pathlib import Path
import tifffile as tiff
import numpy as np
import matplotlib.pyplot as plt

# Ensure the folder exists
if not dataset_path.is_dir():
    raise FileNotFoundError(f"The folder {DATASET_FOLDER} does not exist.")

# Find all .tif files in the folder (including subdirectories if needed)
tif_files = list(dataset_path.rglob("*.tif"))  # Use `rglob` for recursive search

# Check if any .tif files were found
if not tif_files:
    print("No .tif files found in the specified folder.")
else:
    print(f"Found {len(tif_files)} .tif file(s).")

histograms = []
bin_edges = []

# Process each .tif file
for file_path in tif_files:
    print(f"Processing file: {file_path}")
    try:
        # Read the TIFF file
        with tiff.TiffFile(file_path) as tif:
            # Load the TIFF data (all frames)
            data = tif.asarray()
            print(f"Loaded file {file_path} with shape {data.shape} and dtype {data.dtype}")
            
            # Calculate and plot the histogram
            title = f"Histogram for {file_path.name}"
            hist, bins = np.histogram(data, bins=65536, range=(0, 65535))
            histograms.append(hist)
            bin_edges.append(bins)
         

    except Exception as e:
        print(f"Failed to process file {file_path}: {e}")


In [ ]:
hists = np.stack(histograms)
hists

In [ ]:
np.save("test_data_hist", hists)

In [ ]:
# Iterate over each image's histogram
for i in range(hists.shape[0]):
    # Get the histogram for the i-th image
    image_hist = hists[i]
    
    # Find the range of non-zero bins
    non_zero_indices = np.nonzero(image_hist)[0]
    if non_zero_indices.size == 0:
        print(f"Image {i + 1} has no non-zero bins. Skipping.")
        continue
    
    first_non_zero = non_zero_indices[0]
    last_non_zero = non_zero_indices[-1]
    
    # Extract the relevant part of the histogram
    relevant_hist = image_hist[first_non_zero:last_non_zero + 1]
    relevant_bins = np.arange(first_non_zero, last_non_zero + 1)
    
    # Plot the histogram
    plt.figure(figsize=(10, 6))
    plt.bar(relevant_bins, relevant_hist, width=1, color='blue', alpha=0.7)
    plt.title(f"Histogram for Image {i + 1} (Bins {first_non_zero} to {last_non_zero})", fontsize=16)
    plt.xlabel("Pixel Intensity", fontsize=14)
    plt.ylabel("Frequency", fontsize=14)
    plt.grid(alpha=0.4)
    plt.tight_layout()
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

for h in hists[:, :512]:
    plt.figure()
    sns.histplot(h, bins=65536)